In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints
from shared.tools import find_max_clique

%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR)

In [ ]:
l_img, r_img = dataset.get_color_images(20)
n_img, _ = dataset.get_color_images(21)
Q_left = dataset.get_color_left_Q_matrix()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(l_img)
ax2.imshow(r_img)

In [ ]:
vo = VisualOdometry()
depth_frame = vo.process_depth(l_img, r_img, Q_left)

print(depth_frame[0,0])

plt.figure(figsize=(15,9))
plt.imshow(depth_frame[:,:,2])

In [ ]:
def get_feature_3d_points(c_feats, n_feats, depth_frame):
    c_points = []
    for ft in c_feats:
        pnt = depth_frame[ft[1], ft[0]]
        c_points.append(pnt)
    n_points = []
    for ft in n_feats:
        pnt = depth_frame[ft[1], ft[0]]
        n_points.append(pnt)

    c_points = np.array(c_points)
    n_points = np.array(n_points)
    
    ft_idxs = (c_points[:,2] > 0) & (n_points[:,2] > 0)
        
    return c_points[ft_idxs], n_points[ft_idxs], ft_idxs

In [ ]:
c_feats, n_feats = vo.get_features(l_img, n_img)

c_img_canvas = l_img.copy()
n_img_canvas = n_img.copy()

c_pnts, n_pnts, ft_idxs = get_feature_3d_points(c_feats, n_feats, depth_frame)

draw_keypoints(c_img_canvas, n_img_canvas, c_feats[ft_idxs], n_feats[ft_idxs])

# Rendering valid features
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

In [ ]:
import networkx as nx
from networkx.algorithms.approximation.clique import max_clique

num_points = c_pnts.shape[0]
graph = nx.Graph()
graph.add_nodes_from(list(range(num_points)))

print(num_points)
# print(graph.number_of_nodes())
dist_thrs = 0.2

for i in range(num_points):
    diff_1 = c_pnts[i,:] - c_pnts
    diff_2 = n_pnts[i,:] - n_pnts
    dist_1 = np.linalg.norm(diff_1, axis=1)
    dist_2 = np.linalg.norm(diff_2, axis=1)
    diff = abs(dist_2 - dist_1)
    wIdx = np.where(diff < dist_thrs)
    for i_w in wIdx[0]:  
        graph.add_edge(i, i_w)

print(graph.number_of_edges())
print(graph.number_of_nodes())

cliques = nx.algorithms.find_cliques(graph)

max_clique = max_clique(graph)

# for clique in cliques:
#     print(clique)

# print(max_clique)
# print(list(max_clique))
    
# nx.draw(graph)
# plt.show()


c_pnts_new = c_pnts[list(max_clique)]
n_pnts_new = n_pnts[list(max_clique)]

intr_left, _ = dataset.get_color_intrinsics_dicts()

P_left, _ = dataset.get_color_P_matrix()
P_left = np.hstack((P_left, np.transpose([[0, 0, 0]])))
P_left[1,1] *= -1
P_left[0,0] *= -1
print(P_left)

c_pnts_2d = np.ones((c_pnts_new.shape[0], c_pnts_new.shape[1]+1))
c_pnts_2d[:,:3] = c_pnts_new

# n_pnts_2d = n_pnts_new.copy()

print(c_pnts_2d)

# xs = -c_pnts_2d[:,0] * intr_left['fx'] / c_pnts_2d[:,2] + intr_left['cx']
# ys = -c_pnts_2d[:,1] * intr_left['fy'] / c_pnts_2d[:,2] + intr_left['cy']
# c_pnts = np.hstack((np.transpose([xs]), np.transpose([ys])))

# xs = -n_pnts_2d[:,0] * intr_left['fx'] / n_pnts_2d[:,2] + intr_left['cx']
# ys = -n_pnts_2d[:,1] * intr_left['fy'] / n_pnts_2d[:,2] + intr_left['cy']
# n_pnts = np.hstack((np.transpose([xs]), np.transpose([ys])))

# print(pnts.astype(int))

c_pnts_2d = P_left @ np.transpose(c_pnts_2d)
c_pnts_2d = np.transpose(c_pnts_2d)
c_pnts_2d /= c_pnts_2d[:,2:3] # To keep array
c_pnts_2d = c_pnts_2d[:,:2].astype(int)


n_pnts_2d = np.ones((n_pnts_new.shape[0], n_pnts_new.shape[1]+1))
n_pnts_2d[:,:3] = n_pnts_new

n_pnts_2d = P_left @ np.transpose(n_pnts_2d)
n_pnts_2d = np.transpose(n_pnts_2d)
n_pnts_2d /= n_pnts_2d[:,2:3] # To keep array
n_pnts_2d = n_pnts_2d[:,:2].astype(int)
# print(c_pnts_2d)
# c_pnts_2d = np.transpose(c_pnts_2d).astype(int)

# print(c_pnts_2d)

c_img_canvas = l_img.copy()
n_img_canvas = n_img.copy()

draw_keypoints(c_img_canvas, n_img_canvas, c_pnts_2d, n_pnts_2d)

# # Rendering valid features
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)